In [1]:
import os
os.environ["ISISROOT"] = "/usgs/cpkgs/anaconda3_linux/envs/isis3.9.1"
os.environ["ISIS3DATA"] = "/usgs/cpkgs/isis3/data"

from knoten.bundle import *
from knoten.csm import create_csm
from plio.io import io_controlnetwork
import csmapi

import threading
import concurrent.futures
import numpy as np

/Users/ladoramkershner/miniconda3/envs/knoten_jupyter/lib/python3.7/site-packages/ale/__init__.py:22: UserWarning: ALESPICEROOT environment variable not set, Spice Drivers will not function correctly
  warnings.warn('ALESPICEROOT environment variable not set, Spice Drivers will not function correctly')


# Distributed Bundle Adjustment

In [2]:
def update_single_sensor(sensor, updates, parameters):
    if len(updates) != len(parameters):
        print("Updates and parameters are of different lengths!")
        return
    for param_idx in range(len(parameters)):
        current_value = sensor.getParameterValue(parameters[param_idx].index)
        sensor.setParameterValue(parameters[param_idx].index, current_value + updates[param_idx])
        
def compute_single_residual(sensor, ground_pt, measure):
    ground_pt = csmapi.EcefCoord(ground_pt[0], ground_pt[1], ground_pt[2])
    img_coord = sensor.groundToImage(ground_pt)
    return measure - np.array([img_coord.line, img_coord.samp])

In [3]:
def admm_iteration(point_id, serial, sensor, 
                    parameters, measure,
                    r, s, rho_x, rho_y, 
                    image_weights, point_weights, 
                    x_old, y_old,
                    threshold=1e-7, max_its=20):
    """
    Parameters
    ----------
    point_id:  str
               network id for point
               
    serial:    str
               serial number associated with the measure's images
                  
    sensor:    csmapi.csmapi.RasterGM
               sensor model
    
    parameters: list
                list of CsmParameter objects representing sensor solve parameters
                
    ground_pt:  numpy.ndarray
                Adjusted X, Y, Z coordinates of ground point
                
    measure:    numpy.ndarray
                line and sample of measure
            
    r:  numpy.ndarray
    
    s:  numpy.ndarray
    
    rho_x:  float
    
    rho_y:  float
    
    image_weights: numpy.ndarray
    
    point_weights: numpy.ndarray
    
    x_old:  numpy.ndarray
    
    y_old:  numpy.ndarray
    
    thresdhold:  float
                 convergence threshold for G-N 
    
    max_its:  int
              max number of iterations to try for convergence
              
    Returns
    -------
    point_id: str
    
    serial:   str
    
    x_corrections:  np.ndarray
                 point updates
    
    y_corrections:  np.ndarray
                 sensor parameter updates
                  
    """
    # Function
    x_corrections = np.zeros(3)
    y_corrections = np.zeros(len(parameters))

    for i in range(max_its):
        measure_v = compute_single_residual(sensor, x_old, measure)
        point_partials = compute_ground_partials(sensor, x_old)
        sensor_partials = compute_sensor_partials(sensor, parameters, x_old)
        measure_partials = np.concatenate([point_partials, sensor_partials], axis=-1)
        N = measure_partials.T.dot(measure_partials)
        N[:3,:3] += rho_x * np.eye(point_partials.shape[-1]) + point_weights
        N[3:,3:] += rho_y * np.eye(sensor_partials.shape[-1]) + image_weights
        RHS = measure_partials.T.dot(measure_v)
        RHS[:3] -= rho_x * x_old + point_weights.dot(x_corrections) + r
        RHS[3:] -= rho_y * y_old + image_weights.dot(y_corrections) + s

        update = np.linalg.inv(N).dot(RHS)

        x_corrections += update[:3]
        y_corrections += update[3:]

        x_old += update[:3]
        update_single_sensor(sensor, update[3:], parameters)

        if np.linalg.norm(update) < threshold:
            break
    
    return point_id, serial, x_corrections, y_corrections

In [4]:
cubes = 'data/cubes.lis'
sensors_dict = generate_sensors(cubes)

network = 'data/hand_dense.net'
cnet = io_controlnetwork.from_isis(network)
cnet = compute_apriori_ground_points(cnet, sensors_dict) # autoseed did not generate ground points, calculate and repopulate the data frame

all_parameters = {sn: get_sensor_parameters(sensor) for sn, sensor in sensors_dict.items()}
solve_parameters = {sn: params[6:12] for sn, params in all_parameters.items()}

# initial estimate of sigma0
column_dict = compute_coefficient_columns(cnet, sensors_dict, solve_parameters)
num_observations = 2 * len(cnet)
num_parameters = max(col_range[1] for col_range in column_dict.values())
W_observations = np.eye(num_observations) # this is a place holder until Jesse adds his calculations
W_params = compute_parameter_weights(cnet, sensors_dict, solve_parameters, column_dict)

V = compute_residuals(cnet, sensors_dict)
dX = np.zeros(W_params.shape[0])
sigma0 = compute_sigma0(V, dX, W_params, W_observations)
print(sigma0)

4.204919720360203


In [5]:
# z_i,j             -> apriori line/sample - constant
# f(xij(k), yji(k)) -> updated_sensor.ground_to_image(updated_point)

# xij(k)            -> dX of previous iteration (point parameters)
# yji(k)            -> dX of precious iteration (sensor parameters)

# xi(k)             -> consensus variable of point
# yj(k)             -> consensus variable of sensor

# rij(k)            -> Lagrangian multiplier of previous iteration (point parameters)
# sji(k)            -> Lagrangian multiplier of previous iteration (sensor parameters)

# rho_x(k)          -> penalty parameter of the previous iteration (point parameters)
# rho_y(k)          -> penalty parameter of the previous iteration (sensor parameters)


In [6]:
# Initial Bundle Conditions
ptids = cnet['id']
serialnumbers = cnet['serialnumber']

zij = [np.array([l, s]) for l, s in zip(cnet['line'], cnet['sample'])]

rhox = 0.001*np.ones(len(cnet)) # initial estimates of the penatly parameters (point)
rhoy = 0.001*np.ones(len(cnet)) # (camera)

# initial estimates of the lagrangian multipliers
rij_initial = {ptid: np.random.rand(1,3)[0] for ptid in ptids.unique()}
rij = [rij_initial[ptid] for ptid in ptids] # points
sji_initial = {sn: np.round(10*np.random.rand(1,len(solve_parameters[sn]))[0]) for sn in serialnumbers.unique()}
sji = [sji_initial[sn] for sn in serialnumbers] # cameras 

xi = [np.array(np.zeros(3)) for _ in range(0, len(cnet))] # consenseous variable (point)
yj = [np.array(np.zeros(len(solve_parameters[sn]))) for sn in serialnumbers] # consenseous variable (camera)

xij = [np.array(row[["adjustedX", "adjustedY", "adjustedZ"]].values.astype('float64')) for _, row in cnet.iterrows()]
yji = list()
for sn in serialnumbers:
    arr  = np.array([])
    for parameter in solve_parameters[sn]:
        arr = np.append(arr, parameter.value)
    yji.append(arr)

sensors_list = [sensors_dict[sn] for sn in serialnumbers]
parameters_list = [solve_parameters[sn] for sn in serialnumbers]
vtpv_list = [0 for _ in range(0, len(cnet))]

current = pd.DataFrame(columns=['id', 'serialnumber', 'sensor', 'parameters', 'zij', 'rhox', 'rhoy', 'rij', 'sji', 'xij', 'yji', 'xi', 'yj', 'vtpv'], \
                       data=zip(ptids, serialnumbers, sensors_list, parameters_list, zij, rhox, rhoy, rij, sji, xij, yji, xi, yj, vtpv_list))

In [7]:
print(len(current))
current.head()

130


,id,serialnumber,sensor,parameters,zij,rhox,rhoy,rij,sji,xij,yji,xi,yj,vtpv
0,autoseed_001,MRO/CTX/1085197697:073,<csmapi.csmapi.RasterGM; proxy of <Swig Object...,"[6 Omega Bias (2): 0.0 m, 7 Phi Bias (2): 0.0 ...","[302.06913127502867, 200.85677699800772]",0.001,0.001,"[0.7545983067809806, 0.30226941011733066, 0.10...","[7.0, 1.0, 5.0, 5.0, 8.0, 1.0]","[2919361.3500983184, -1194116.8210146269, 1238...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0
1,autoseed_001,MRO/CTX/1157902986:250,<csmapi.csmapi.RasterGM; proxy of <Swig Object...,"[6 Omega Bias (2): 0.0 m, 7 Phi Bias (2): 0.0 ...","[15166.790917003123, 744.1819182946748]",0.001,0.001,"[0.7545983067809806, 0.30226941011733066, 0.10...","[9.0, 7.0, 2.0, 9.0, 2.0, 3.0]","[2919361.3500983184, -1194116.8210146269, 1238...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0
2,autoseed_001,MRO/CTX/1096561308:045,<csmapi.csmapi.RasterGM; proxy of <Swig Object...,"[6 Omega Bias (2): 0.0 m, 7 Phi Bias (2): 0.0 ...","[13311.618299705062, 406.4429423017374]",0.001,0.001,"[0.7545983067809806, 0.30226941011733066, 0.10...","[1.0, 2.0, 9.0, 4.0, 2.0, 8.0]","[2919361.3500983184, -1194116.8210146269, 1238...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0
3,autoseed_002,MRO/CTX/1085197697:073,<csmapi.csmapi.RasterGM; proxy of <Swig Object...,"[6 Omega Bias (2): 0.0 m, 7 Phi Bias (2): 0.0 ...","[95.88393113304467, 1925.9642063681222]",0.001,0.001,"[0.18048150856645673, 0.7133177098398927, 0.63...","[7.0, 1.0, 5.0, 5.0, 8.0, 1.0]","[2923282.5660719196, -1184949.09107713, 123813...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0
4,autoseed_002,MRO/CTX/1157902986:250,<csmapi.csmapi.RasterGM; proxy of <Swig Object...,"[6 Omega Bias (2): 0.0 m, 7 Phi Bias (2): 0.0 ...","[14937.449494029423, 2092.5731615034165]",0.001,0.001,"[0.18048150856645673, 0.7133177098398927, 0.63...","[9.0, 7.0, 2.0, 9.0, 2.0, 3.0]","[2923282.5660719196, -1184949.09107713, 123813...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0


In [ ]:
# Bundle
iteration = 0
max_iterations = 10
simga0_diff_tol = 1e-10
update_threshold = 1e-7
for i in range(max_iterations):
    print(f'Iteration {iteration}: ')
    iteration += 1

    # transition old values - may not need?
    previous = current
    old_sigma0 = sigma0
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        results = list()
        for i, row in current.iterrows():
            point_id = row['id']
            serial = row['serialnumber']
            sensor = row['sensor']
            parameters = row['parameters']
            apriori_measure = row['zij']
            image_weights = compute_image_weight(sensor, parameters)
            point_weights = compute_point_weight(cnet, point_id)
#             print('point_weights', point_weights)
            adjusted_gp = row['xij'] # adjusted ground points
            adjusted_sp = row['yji'] # adjusted sensor parameter values

            results.append(executor.submit(admm_iteration, \
                                           point_id, serial, sensor, \
                                           parameters, apriori_measure, \
                                           row['rij'], row['sji'], row['rhox'], row['rhoy'], \
                                           image_weights, point_weights, \
                                           adjusted_gp, adjusted_sp, \
                                           threshold=1e-7, max_its=20))
            
            
        for f in concurrent.futures.as_completed(results):
            # results structured like (ptid, sn, xij (point), yji (camera))
            result = f.result()
            ptid = result[0]
            sn = result[1]
            
            # save new point and sensors states
            index = current[(current['serialnumber'] == sn) & (current['id'] == ptid)].index[0]
            adjusted_gp = current.loc[index, 'xij']+result[2]
            adjusted_sp = current.loc[index, 'yji']+result[3]
            current.loc[index, 'xij':'yji'] = adjusted_gp, adjusted_sp
            
            # apply the sensor corrections to the sensor and the parameter.values
            parameters = current.loc[index, 'parameters']
            update_single_sensor(current.loc[index, 'sensor'], result[3], parameters)
            for i in range(0, len(parameters)):
                current.loc[index, 'parameters'][i].value += result[3][i]
                
            # calculate portion of vtpv - may not need?
            sensor = current.loc[index, 'sensor']
            parameters = current.loc[index, 'parameters']
            ground_pt = current.loc[index, 'xij']
            measure = current.loc[index, 'zij']
            image_weights = compute_image_weight(sensor, parameters)
            point_weights = compute_point_weight(cnet, point_id)
            measure_v = compute_single_residual(sensor, ground_pt, measure)
            c = image_weights.shape[0]
            p = point_weights.shape[0]
            param_weights = np.block([[point_weights, np.zeros((p, c))],
                                     [np.zeros((c,p)), image_weights]])
            dX = np.hstack([result[2], result[3]])
            current.loc[index, 'vtpv'] = measure_v.dot(measure_v) + dX.dot(param_weights).dot(dX)
            
            
    # Calculate consenseous variables
    for sn, group in current.groupby('serialnumber'):
        # calculate consenseous variable
        xij_sum = sum(group['xij'])
        rhox = np.array(group['rhox'])
        rij = np.array(group['rij'])
        constraint_sum = sum((1/rhox)*rij)
        xi = (1/len(group))*(xij_sum + constraint_sum)
        # update concenseous variable
        indices = list(group.index)
        for index in indices:
            current.at[index, 'xi'] = xi
    for pt, group in current.groupby('id'):
        # calculate consenseous variable
        yji_sum = sum(group['yji'])
        rhoy = np.array(group['rhoy'])
        sji = np.array(group['sji'])
        constraint_sum = sum((1/rhoy)*sji)
        yj = (1/len(group))*(yji_sum + constraint_sum)
        # update concenseous variable
        indices = list(group.index)
        for index in indices:
            current.at[index, 'yj'] = yj
            
    for i, row in current.iterrows():
        # update lagrangian multiplier estimates
        rij = row['rij'] + row['rhox']*(row['xij']-row['xi'])
        sji = row['sji'] + row['rhoy']*(row['yji']-row['yj'])
        current.loc[i, 'rij':'sji'] = rij, sji
        # update penalty parameters
        rhox = row['rhox']*10
        rhoy = row['rhoy']*10
        
    # Check on full bundle convergence - two methods presented here
    # 1.) sum all VTPV and divided by (num_obs - num_param)
    sigma0 = np.sqrt(current['vtpv'].sum()/(num_observations-num_parameters))
    print('sigma0 = ', sigma0)
    if (abs(sigma0 - old_sigma0) < simga0_diff_tol):
        print(f'change in sigma0 of {abs(sigma0 - old_sigma0)} - Converged!')
        break
    # 2.) take norm consensus variable update
#     update = 
#     if (np.linalg.norm(update) < update_threshold):
#         print(f"update nrom of {np.linalg.norm(update)} - Converged!")
#         break

Iteration 0: 
sigma0 =  7806.3589905018825
Iteration 1: 
